In [4]:
import os
import torch
from pathlib import Path
from ultralytics import YOLO

def convert_folder_to_torchscript(input_folder, output_folder):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_folder, exist_ok=True)
    
    # Tìm tất cả file .pt trong thư mục đầu vào
    pt_files = [f for f in os.listdir(input_folder) if f.endswith('.pt')]
    
    print(f"Tìm thấy {len(pt_files)} file .pt trong thư mục {input_folder}")
    
    for pt_file in pt_files:
        input_path = os.path.join(input_folder, pt_file)
        output_name = os.path.splitext(pt_file)[0] + '.torchscript.pt'
        output_path = os.path.join(output_folder, output_name)
        
        print(f"Đang xử lý: {pt_file}")
        
        try:
            # Cách 1: Sử dụng phương thức export tích hợp
            model = YOLO(input_path)
            model.export(format="torchscript", imgsz=640, simplify=True)
            
            # Tìm file xuất ra (thường trong thư mục runs/detect/train/weights/)
            export_path = Path("runs/export") / f"{os.path.splitext(pt_file)[0]}_torchscript.pt"
            if export_path.exists():
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                os.replace(str(export_path), output_path)
                print(f"Đã lưu mô hình TorchScript vào: {output_path}")
            else:
                print(f"Không tìm thấy file xuất: {export_path}")
                
        except Exception as e:
            print(f"Lỗi khi sử dụng export: {e}")
            
            try:
                # Cách 2: Sử dụng trace thay vì script
                print(f"Thử phương pháp trace cho {pt_file}...")
                model = YOLO(input_path)
                traced_model = model.model.eval().cpu()
                
                # Tạo input mẫu cho trace
                example_input = torch.rand(1, 3, 640, 640)
                
                # Áp dụng trace
                with torch.no_grad():
                    traced_model = torch.jit.trace(traced_model, example_input)
                    
                # Lưu mô hình
                traced_model.save(output_path)
                print(f"Đã lưu mô hình TorchScript (trace) vào: {output_path}")
            except Exception as e2:
                print(f"Lỗi khi sử dụng trace: {e2}")
    
    print("Chuyển đổi hoàn tất")

# Chỉ định thư mục đầu vào và đầu ra
input_folder = r"C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n"
output_folder = r"C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\models\torchscript"

# Thực hiện chuyển đổi
convert_folder_to_torchscript(input_folder, output_folder)

Tìm thấy 6 file .pt trong thư mục C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n
Đang xử lý: bamdinh.pt
Ultralytics 8.3.149  Python-3.10.16 torch-2.7.0+cpu CPU (Intel Core(TM) i7-14700K)
 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv12n summary (fused): 159 layers, 2,556,923 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\bamdinh.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.3 MB)

TorchScript: starting export with torch 2.7.0+cpu...
TorchScript: export success  1.7s, saved as 'C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\bamdinh.torchscript' (10.5 MB)

Export complete (2.1s)
Results saved to C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n
Predict:         yolo predict task=detect model=C:\Users\1\MeikoAI\SonCode\TestCycletimeMeiko\ModelYolov12n\ba